In [39]:
from basic_tools import *
from config import DockerConfig
from media import Media
from openai_api import GPTApi, WisperApi
from subtitle import SubtitleWriter, SubtitleDecoder
from media import Media

cfg = DockerConfig()
gpt = GPTApi(cfg("OPENAI_API_KEY_PETER"))
wis = WisperApi(cfg("OPENAI_API_KEY_PETER"))



读取本地配置文件: config.ini


In [40]:
md = Media(FilePath(r"C:\Users\lucyc\Desktop\How Much Muscle Effort Do I Put Into The Exercises_ - YouTube - Google Chrome 2024-05-02 15-40-54.mp4"),
           SystemCmd())

In [3]:
res = md.get_audio_pice(30)

ffmpeg -i "C:\Users\lucyc\Desktop\How Much Muscle Effort Do I Put Into The Exercises_ - YouTube - Google Chrome 2024-05-02 15-40-54.mp4" -ss 0 -t 30 -aq 0 -map a "C:\Users\lucyc\Desktop\temp.mp3"


In [4]:
audio_file = open(r"C:\Users\lucyc\Desktop\temp.mp3", "rb")
transcript = wis.transcribe_timestamp(audio_file)

In [5]:
print(transcript)

Transcription(text="face aerobics to go and have a look at that video. In that video it shows you a number of things. The first thing it shows you and teaches you is how much muscle effort to put into the exercises. So when I do the exercises you can see me just lifting my forehead, you can see me just lifting my cheeks, you can see me just going like this. Okay and as I do those things I'm squeezing the muscle.", task='transcribe', language='english', duration=30.0, words=[{'word': 'face', 'start': 0.0, 'end': 0.47999998927116394}, {'word': 'aerobics', 'start': 0.47999998927116394, 'end': 0.9200000166893005}, {'word': 'to', 'start': 0.9200000166893005, 'end': 1.440000057220459}, {'word': 'go', 'start': 1.440000057220459, 'end': 1.6399999856948853}, {'word': 'and', 'start': 1.6399999856948853, 'end': 2.0}, {'word': 'have', 'start': 2.0, 'end': 2.0199999809265137}, {'word': 'a', 'start': 2.0199999809265137, 'end': 2.259999990463257}, {'word': 'look', 'start': 2.259999990463257, 'end': 2

In [6]:
words = transcript.words
print(words)

[{'word': 'face', 'start': 0.0, 'end': 0.47999998927116394}, {'word': 'aerobics', 'start': 0.47999998927116394, 'end': 0.9200000166893005}, {'word': 'to', 'start': 0.9200000166893005, 'end': 1.440000057220459}, {'word': 'go', 'start': 1.440000057220459, 'end': 1.6399999856948853}, {'word': 'and', 'start': 1.6399999856948853, 'end': 2.0}, {'word': 'have', 'start': 2.0, 'end': 2.0199999809265137}, {'word': 'a', 'start': 2.0199999809265137, 'end': 2.259999990463257}, {'word': 'look', 'start': 2.259999990463257, 'end': 2.2799999713897705}, {'word': 'at', 'start': 2.2799999713897705, 'end': 2.440000057220459}, {'word': 'that', 'start': 2.440000057220459, 'end': 2.5999999046325684}, {'word': 'video', 'start': 2.5999999046325684, 'end': 3.0199999809265137}, {'word': 'In', 'start': 4.460000038146973, 'end': 4.539999961853027}, {'word': 'that', 'start': 4.539999961853027, 'end': 4.760000228881836}, {'word': 'video', 'start': 4.760000228881836, 'end': 5.0}, {'word': 'it', 'start': 5.0, 'end': 5.

In [7]:
sentence = ""
for word in words:
    print(word["word"], end=" ")
    sentence += word["word"] + " "
sentence = sentence[:-1]


face aerobics to go and have a look at that video In that video it shows you a number of things The first thing it shows you and teaches you is how much muscle effort to put into the exercises So when I do the exercises you can see me just lifting my forehead you can see me just lifting my cheeks you can see me just going like this Okay and as I do those things I'm squeezing the muscle 

In [53]:
prompt = """
You are a subtitle segmenter. Your task is to segment the text enclosed in <<<>>> symbols into short sections based on semantics and pauses, with each section not exceeding 12 words and containing no punctuation. Each segmented subtitle should be presented in the following format:
-
subtitle text 1
-
subtitle text 2
-
Maintain the natural flow of the dialogue and ensure each segment can independently convey a complete meaning. Do not provide any explanations with your outputs. Do not modify any text.

"""

In [54]:
usermsg = "<<<{}>>>".format(sentence)

In [55]:
# openAI message format
message = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": usermsg}
]
message

[{'role': 'system',
  'content': '\nYou are a subtitle segmenter. Your task is to segment the text enclosed in <<<>>> symbols into short sections based on semantics and pauses, with each section not exceeding 12 words and containing no punctuation. Each segmented subtitle should be presented in the following format:\n-\nsubtitle text 1\n-\nsubtitle text 2\n-\nMaintain the natural flow of the dialogue and ensure each segment can independently convey a complete meaning. Do not provide any explanations with your outputs. Do not modify any text.\n\n'},
 {'role': 'user',
  'content': "<<<face aerobics to go and have a look at that video In that video it shows you a number of things The first thing it shows you and teaches you is how much muscle effort to put into the exercises So when I do the exercises you can see me just lifting my forehead you can see me just lifting my cheeks you can see me just going like this Okay and as I do those things I'm squeezing the muscle>>>"}]

In [56]:
response = gpt.query(message, max_tokens=1000, temperature=0, model="gpt-4o")

In [57]:
res_sentences = response[2:-2].split("\n-\n")

In [58]:
res_sentences

['face aerobics to go and have a look',
 'at that video In that video it',
 'shows you a number of things The first',
 'thing it shows you and teaches you is',
 'how much muscle effort to put into',
 'the exercises So when I do the',
 'exercises you can see me just lifting',
 'my forehead you can see me just',
 'lifting my cheeks you can see me',
 'just going like this Okay and as',
 "I do those things I'm squeezing the",
 'muscle']

In [59]:
nres = " ".join(res_sentences)
if len(nres) != len(sentence):
    print("Lengths do not match")
else:
    for i in range(len(nres)):
        if nres[i] == sentence[i]:
            pass
        else:
            print(nres[i], sentence[i])
    print("Check complete") 


Check complete


In [60]:
orig_word_list = sentence.split()
orig_index = 0
time_stamps = []
sentences = []

start = words[orig_index]["start"]
for res_sent in res_sentences:

    orig_index += len(res_sent.split())
    if orig_index+1 > len(words):
        orig_index -= 1

    end = words[orig_index]["end"]
    time_stamps.append((start, end))
    sentences.append(res_sent)
    
    start = words[orig_index]["start"]



In [61]:
time_stamps

[(0.0, 2.440000057220459),
 (2.2799999713897705, 5.539999961853027),
 (5.539999961853027, 8.15999984741211),
 (8.020000457763672, 10.180000305175781),
 (10.0, 11.9399995803833),
 (11.619999885559082, 15.079999923706055),
 (14.720000267028809, 17.5),
 (16.860000610351562, 21.239999771118164),
 (20.899999618530273, 24.600000381469727),
 (24.420000076293945, 28.31999969482422),
 (28.15999984741211, 29.979999542236328),
 (29.8799991607666, 29.979999542236328)]

In [62]:
sentences

['face aerobics to go and have a look',
 'at that video In that video it',
 'shows you a number of things The first',
 'thing it shows you and teaches you is',
 'how much muscle effort to put into',
 'the exercises So when I do the',
 'exercises you can see me just lifting',
 'my forehead you can see me just',
 'lifting my cheeks you can see me',
 'just going like this Okay and as',
 "I do those things I'm squeezing the",
 'muscle']

In [63]:
def time_convert(time):
    hours = time // 3600
    remain = time % 3600
    min = remain // 60
    remain = remain % 60
    sec = remain // 1
    msec = remain % 1
    msec = (msec // 0.001)
    return (int(hours),int(min),int(sec),int(msec))

In [64]:
time_convert(time_stamps[5][0])

(0, 0, 11, 619)

In [ ]:
class WHISPER_DRIVER:

    def __init__(self) -> None:
        self.model = None
        self.result = None
        self.num_of_sentence = 0
        self.sentence_index = 0

    def model_load(self,model = "tiny.en"):
        self.model = whisper.load_model(model)
     
    def transcribe(self,file_path):
        self.result = self.model.transcribe(file_path,verbose=True)
    
    def get_result(self):
        return self.result

    def init_reader(self):
        self.num_of_sentence = len(self.result["segments"])
        self.sentence_index = -1
    
    def time_convert(self,time):
        hours = time // 3600
        remain = time % 3600
        min = remain // 60
        remain = remain % 60
        sec = remain // 1
        msec = remain % 1
        msec = (msec // 0.001)
        return (int(hours),int(min),int(sec),int(msec))

    def get_sentence(self):
        self.sentence_index += 1
        if self.sentence_index < self.num_of_sentence:
            return (self.time_convert(self.result["segments"][self.sentence_index]["start"]),\
                    self.time_convert(self.result["segments"][self.sentence_index]["end"]),\
                    self.result["segments"][self.sentence_index]["text"])
        else:
            return (None,None,None)
        

In [1]:
def build_shift_table(pattern):
    """构建移位表，用于跳过尽可能多的无效字符位置"""
    m = len(pattern)
    table = {ch: m for ch in pattern}
    for i in range(m - 1):
        table[pattern[i]] = m - 1 - i
    return table

In [7]:
tb = build_shift_table("hello")
print(tb)

{'h': 4, 'e': 3, 'l': 1, 'o': 5}


In [ ]:


"""ArithmeticError

"""